In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

import time

# URL 분석
- 카테고리 페이지는 `middle_category_id`라는 인자를 통해 결정된다.
    - [상의](https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474): `474`
    - [하의 (팬츠)](https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=547): `547`
- 추후 카테고리가 추가될 때의 편의성을 위해, `dict`에 카테고리 아이디를 담아 넣는다.
    - 필요하다면, 이를 config 파일로 빼내기
- `리뷰많은순`으로 정렬 순서를 바꿀 필요가 있는데, 이는 맨 뒤에 `sort` 인자에 `201`을 넣어주면 해결된다.

In [7]:
zigzag_url = "https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id={id}&sort=201"
id_dict = {
    '상의' : '474',
    '하의' : '547'
}

for category, id in id_dict.items():
    print(category, ":", zigzag_url.format(id=id))

상의 : https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474&sort=201
하의 : https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=547&sort=201


# 크롤링 순서
1. 정렬 순서를 `리뷰많은순`으로 변경한다.
    - URL로 정렬 방식 변경 가능
2. 상품 링크를 원하는 수량만큼 불러온다. (테스트의 경우 시간 문제로 10개만)

3. 링크에 들어간다.
4. 상품과 관련된 데이터 크롤링
    - 상품 이름
    - 상품 가격 (할인 제외가격)
    - 카테고리 (`id_dict`의 key)
    - 이미지 url (한장)
    - 상세설명 (우선 스킵)
    - 사이즈 정보 (리스트로)
5. 상품 리뷰와 관련된 데이터 크롤링
    - 리뷰아이디
    - 성별, 키, 몸무게
    - 구매 관련 정보
    - 핏 관련 정보
    - 기타 후기


## 상품 링크 불러오기

In [64]:
def get_product_links(driver, url, max_num=2):
    link_set = set()
    link_list = []
    wait = WebDriverWait(driver, 10)
    driver.get(url)
    xpath = '//*[@id="__next"]/div[1]/div/main/section[2]/div/div/div/div/div[1]/div/div[{i}]/div/a'
    i = 1
    while len(link_set) < max_num:
        try:
            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath.format(i=i))))
            href = element.get_attribute('href')
            if href not in link_set:
                link_list.append(href)
            link_set.add(href)
            i += 1
        except (NoSuchElementException, TimeoutException) as e:
            print(e)
            continue

    return link_list

In [65]:
## get_product_links test
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
for _, id in id_dict.items():
    url = zigzag_url.format(id=id)
    links = get_product_links(driver, url, 3)
    print(links)
    driver.implicitly_wait(1)
    print()
driver.close()

['https://zigzag.kr/catalog/products/112538672', 'https://zigzag.kr/catalog/products/106711935', 'https://zigzag.kr/catalog/products/100511186']

['https://zigzag.kr/catalog/products/140751630', 'https://zigzag.kr/catalog/products/113041967', 'https://zigzag.kr/catalog/products/100341594']



## 링크 진입 후, 원하는 데이터 크롤링
- XPATH 정리
    - 상품 이름
        - `//*[@id="__next"]/div[1]/div/div/div[4]/h1`
    - 상품 가격 (할인 제외가격)
        - `//*[@id="__next"]/div[1]/div/div/div[6]/div[1]/div/div[1]/div[2]`
    - 카테고리 (`id_dict`의 key)
    - 이미지 url (한장)
        - `//*[@id="__next"]/div[1]/div/div/div[1]/div[1]/div/div/div[1]/div[1]/div/div/picture/img`
    - 상세설명 (우선 스킵)
    - 사이즈 정보 (리스트로)
        - 색상별로 사이즈가 다 달라서 고민이 필요
    - 색상 정보

In [71]:
## 우선은 이름, 가격, 이미지url만
def extract_product_info(driver, url):
    try:
        driver.get(url)
        # driver.implicitly_wait(5)
        # name = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div/div/div[4]/h1')
        wait = WebDriverWait(driver, 10)
        name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'BODY_15 REGULAR css-1qjogoj e1wgb8lp0')))
        # price = wait.until(EC.presence_of_element_located((\
        #     By.XPATH, '//*[@id="__next"]/div[1]/div/div/div[6]/div[1]/div/div[1]/div[2]')))
        # image_url = wait.until(EC.presence_of_element_located((\
        #     By.XPATH, '//*[@id="__next"]/div[1]/div/div/div[1]/div[1]/div/div/div[1]/div[1]/div/div/picture/img')))

        return {
            'name': name.text,
            # 'price': price.get_attribute('div'),
            # 'image_url': image_url.get_attribute('')
        }
    except (NoSuchElementException, TimeoutError) as e:
        print(e)
    finally:
        driver.close()

In [74]:
## 지그재그 리뷰는 일단 되는듯

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/review/list/111217060")
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    
    wait = WebDriverWait(driver, 5)

    # p = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-1r5ocs1 ej5yuks0')))

    for i in range(1, 11):
        p = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[1]/div/div/div/div[2]/div/div/section/div[{i}]/div[1]/div[3]/div[1]')))
        

옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
159cm39kg상의 44
옵션
핑크(민자)Free 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
163cm52kg상의 44
옵션
블랙(민자)Free 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
161cm63kg상의 66
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
165cm53kg상의 55
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
보통이에요
색감
화면과 비슷해요
정보
164cm53kg상의 55
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
158cm54kg상의 55
옵션
[TT037] 보들쫀쫀 끈나시_화이트(골지)FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
160cm54kg상의 66
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
정보
165cm54kg상의 66
옵션
[TT037] 보들쫀쫀 끈나시_단가라FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요
옵션
[TT037] 보들쫀쫀 끈나시_그레이(골지)FREE 직진
사이즈
정사이즈예요
퀄리티
아주 만족해요
색감
화면과 비슷해요


In [70]:
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/catalog/products/112538672?tab=review")
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    driver.implicitly_wait(1)
    
    wait = WebDriverWait(driver, 10)

    name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'BODY_15.REGULAR.css-1qjogoj.e1wgb8lp0')))
    print(name.text)

    price = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-14j45be.eizm2tm0')))
    print(price.text)

    img_url = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'show-skeleton.css-12ywniv')))
    print(img_url.find_element(By.TAG_NAME, 'picture').find_element(By.TAG_NAME, 'img').get_attribute('src'))

[빅사이즈추가/워터밤][16만장돌파] 옆셔링 캡내장 슬리브리스
41,700
https://cf.product-image.s.zigzag.kr/original/d/2024/6/7/7416_202406071201310448_58975.jpeg?width=400&height=400&quality=80&format=webp


In [7]:
def get_color_list(driver):
    wait = WebDriverWait(driver, 10)
    color_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-0.e1u2d7n04')))
    return color_table.find_elements(By.TAG_NAME, 'li')

def get_size_list(driver):
    wait = WebDriverWait(driver, 10)
    return wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'css-1ef2f7p.ehwmh5c10')))

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/catalog/products/112538672")
    # driver.get("https://zigzag.kr/catalog/products/111217060?tab=review")

    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    time.sleep(1)  # 암시적 대기 대신 명시적 대기 사용 권장

    purchase_button = driver.find_element(By.CLASS_NAME, 'BODY_17.BOLD.css-hbld7k.e1yh52zv0')
    ActionChains(driver).click(purchase_button).perform()

    wait = WebDriverWait(driver, 10)
    color_table_button = driver.find_element(By.CLASS_NAME, 'BODY_15.SEMIBOLD.css-utqis4.e1cn5bmz0')

    color_list = get_color_list(driver)

    for i in range(len(color_list)):
        color_list = get_color_list(driver)  # 새로 로드된 DOM에서 색상 리스트 다시 가져오기
        color_tag = color_list[i]
        color = color_tag.text
        color_tag.click()
        time.sleep(1)

        size_list = get_size_list(driver)
        print(color)

        for size_tag in size_list:
            size_info = size_tag.text
            time.sleep(2)
            size_info = size_info.split("\n")
            size = size_info[0]
            price = size_info[1]
            print("\t", size, price)

        # 색상 테이블 버튼 클릭하여 색상 선택 창 다시 열기
        color_table_button.click()
        time.sleep(1)

아이보리
	 one size 13,900원
	 L 17,900원
핑크
	 one size 13,900원
소라
	 one size 13,900원
블랙
	 one size 13,900원
	 L 17,900원
(롱ver)아이보리
	 one size 14,900원
(롱ver)핑크
	 one size 14,900원
(롱ver)소라
	 one size 14,900원
(롱ver)블랙
	 one size 14,900원
그레이
	 one size 13,900원
(롱ver)그레이
	 one size 14,900원
크림
	 one size 13,900원
(롱ver)크림
	 one size 14,900원
(롱롱ver)아이보리
	 one size 15,900원
(롱롱ver)블랙
	 one size 15,900원
백멜란지
	 one size 13,900원
(롱ver)코발트블루
	 one size 14,900원
(롱ver)백멜란지
	 one size 14,900원


## 스크롤 테스트
- CSS_SELECTOR를 통해 원하는 곳으로 이동

In [9]:
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474&sort=201")
    action = ActionChains(driver)
    time.sleep(1)

    wait = WebDriverWait(driver, 5)
    next_raw = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-index="2"]')))

    action.move_to_element(next_raw).perform()
    time.sleep(15)


In [3]:
## 실제 사용 예시
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474&sort=201")
    action = ActionChains(driver)
    time.sleep(1)
    wait = WebDriverWait(driver, 10)
    
    for i in range(10):
        next_raw = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f'div[data-index="{i}"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_raw)
        action.move_to_element_with_offset(next_raw, 0, 100).perform()
        raw = next_raw.find_elements(By.CLASS_NAME, 'css-1jo7xgn')
        driver.implicitly_wait(1)
        for div in raw:
            a = div.find_element(By.XPATH, './/div/a')
            href = a.get_attribute('href')
            print(href, end=', ')
        print()
        time.sleep(1)


https://zigzag.kr/catalog/products/112538672, https://zigzag.kr/catalog/products/106711935, https://zigzag.kr/catalog/products/100511186, 
https://zigzag.kr/catalog/products/112059662, https://zigzag.kr/catalog/products/111217060, https://zigzag.kr/catalog/products/100309028, 
https://zigzag.kr/catalog/products/143458163, https://zigzag.kr/catalog/products/100949076, https://zigzag.kr/catalog/products/132082569, 
https://zigzag.kr/catalog/products/114626733, https://zigzag.kr/catalog/products/135006673, https://zigzag.kr/catalog/products/106642729, 
https://zigzag.kr/catalog/products/103669774, https://zigzag.kr/catalog/products/131506009, https://zigzag.kr/catalog/products/107705764, 
https://zigzag.kr/catalog/products/114230530, https://zigzag.kr/catalog/products/122210777, https://zigzag.kr/catalog/products/125789213, 
https://zigzag.kr/catalog/products/100485995, https://zigzag.kr/catalog/products/100452288, https://zigzag.kr/catalog/products/111905990, 
https://zigzag.kr/catalog/p

In [14]:
## 실제 사용 예시
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://zigzag.kr/catalog/products/100341594?tab=size")
    wait = WebDriverWait(driver, 10)
    size_list = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'head.CAPTION_11.SEMIBOLD.css-oyuqpv.eqs9ftl0')))
    print(size_list)
    for size in size_list:
        print(size.text)
    


[<selenium.webdriver.remote.webelement.WebElement (session="c5cf381463a5872ea5a69cb3abab13b5", element="f.0FABDA3A476E0151FB8CB50439F9EA74.d.B4A07EEB37062DACADD092835D4A12C8.e.11")>, <selenium.webdriver.remote.webelement.WebElement (session="c5cf381463a5872ea5a69cb3abab13b5", element="f.0FABDA3A476E0151FB8CB50439F9EA74.d.B4A07EEB37062DACADD092835D4A12C8.e.12")>, <selenium.webdriver.remote.webelement.WebElement (session="c5cf381463a5872ea5a69cb3abab13b5", element="f.0FABDA3A476E0151FB8CB50439F9EA74.d.B4A07EEB37062DACADD092835D4A12C8.e.13")>, <selenium.webdriver.remote.webelement.WebElement (session="c5cf381463a5872ea5a69cb3abab13b5", element="f.0FABDA3A476E0151FB8CB50439F9EA74.d.B4A07EEB37062DACADD092835D4A12C8.e.14")>, <selenium.webdriver.remote.webelement.WebElement (session="c5cf381463a5872ea5a69cb3abab13b5", element="f.0FABDA3A476E0151FB8CB50439F9EA74.d.B4A07EEB37062DACADD092835D4A12C8.e.15")>, <selenium.webdriver.remote.webelement.WebElement (session="c5cf381463a5872ea5a69cb3abab13

In [ ]:

# def crawling_color_and_size(driver):
#     color_list = []
#     size_list = []

#     scroll_action = driver.find_element(By.CSS_SELECTOR, 'body')
#     scroll_action.send_keys(Keys.END)
#     time.sleep(1)

#     purchase_button = driver.find_element(By.CLASS_NAME, 'BODY_17.BOLD.css-hbld7k.e1yh52zv0')
#     ActionChains(driver).click(purchase_button).perform()

#     wait = WebDriverWait(driver, 3)
#     color_table_button = driver.find_element(By.CLASS_NAME, 'BODY_15.SEMIBOLD.css-utqis4.e1cn5bmz0')

#     color_tag_list = get_color_tag_list(wait)

#     for i in range(len(color_tag_list)):
#         color_tag_list = get_color_tag_list(wait)  # 새로 로드된 DOM에서 색상 리스트 다시 가져오기
#         color_tag = color_tag_list[i]
#         color = color_tag.text
#         color_tag.click()
#         time.sleep(1)
#         color_list.append(color)

#         size_tag_list = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'css-1ef2f7p.ehwmh5c10')))

#         for size_tag in size_tag_list:
#             size_info = size_tag.text
#             time.sleep(2)
#             size_info = size_info.split("\n")
#             size = size_info[0]
#             price = size_info[1]
#             if size not in size_list:
#                 size_list.append(size)

#         # 색상 테이블 버튼 클릭하여 색상 선택 창 다시 열기
#         color_table_button.click()
#         time.sleep(2)
    
#     return {
#         'color' : color_list,
#         'size' : size_tag_list
#     }


/html/body/div/div[1]/div/div/div/div[2]/div/div/section/div[1]
/html/body/div/div[1]/div/div/div/div[2]/div/div/section/div[2]

In [60]:

def scroll_down(driver):
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    actions.send_keys(Keys.END)
    actions.send_keys(Keys.END)
    actions.send_keys(Keys.END)
    actions.send_keys(Keys.END)
    time.sleep(0.5)

def get_or_none(webelement, xpath, by=By.XPATH):
    try:
        time.sleep(0.1)
        return webelement.find_element(by, xpath).text
    except NoSuchElementException as e:
        return
    
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get('https://zigzag.kr/review/list/106711935')
    wait = WebDriverWait(driver, 3)
    xpath = '/html/body/div/div[1]/div/div/div/div[2]/div/div/section/div[{i}]/div[1]/div[3]'

    for i in range(1, 220):
        time.sleep(0.1)
        x = xpath.format(i=i)
        print(i)
        if i % 4 == 0:
            scroll_down(driver)
        try:
            review_tag = wait.until(EC.presence_of_element_located((By.XPATH, x)))
        except TimeoutException as ee:
            print(i)
            break
        
        time.sleep(1)
    
        selected_color = get_or_none(review_tag, './/div/div[1]/span/span[1]')
        selected_size = get_or_none(review_tag, './/div/div[1]/span/span[2]')
        size_opinion = get_or_none(review_tag, './/div/div[2]/div[1]/span')
        quality_opinion = get_or_none(review_tag, './/div/div[2]/div[2]/span')
        color_opinion = get_or_none(review_tag, './/div/div[2]/div[3]/span')

        height = get_or_none(review_tag, './/div/div[3]/span/span[1]')
        weight = get_or_none(review_tag, './/div/div[3]/span/span[2]')
        top_size = get_or_none(review_tag, './/div/div[3]/span/span[3]')

        detail_text = get_or_none(review_tag, 'BODY_14.REGULAR.css-epr5m6.e1j2jqj72', by=By.CLASS_NAME)

        temp = {
            'selected_color': selected_color,
            'selected_size': selected_size,
            'size_opinion': size_opinion,
            'quality_opinion': quality_opinion,
            'color_opinion': color_opinion,
            'height': height,
            'weight': weight,
            'top_size': top_size,
            'detail_text': detail_text
        }

        print(temp)



1
{'selected_color': '스카이블루(세미크롭ver.)', 'selected_size': 'F(44~66)', 'size_opinion': '정사이즈예요', 'quality_opinion': '아주 만족해요', 'color_opinion': '화면과 비슷해요', 'height': '159cm', 'weight': '58kg', 'top_size': '상의 55', 'detail_text': '159/62/80B 팔뚝살이 넘치고 몸통이 원형인 체형인데 몸에 딱 달라붙는 옷도 싫고 오버핏도 싫어하시는 분들이 입기 적절한 스탠다드 핏 반팔티인것 같아요! 기장도 배꼽에서 반뼘은 더 내려오는 기장이라 세미 크롭 치고도 좀 긴것같아서 부담없이 데일리…\n더보기'}
2
{'selected_color': '아이보리(세미크롭ver.)', 'selected_size': 'F(44~66)', 'size_opinion': '정사이즈예요', 'quality_opinion': '아주 만족해요', 'color_opinion': '화면과 비슷해요', 'height': '164cm', 'weight': '76kg', 'top_size': '상의 77', 'detail_text': '우선 쫀쫀하구요!! 저는 77사이즈임에도 괜찮게 맞아서 정말 좋습니다..ㅎㅎ\n같은 상품 긴팔은 팔뚝부각이 조금 있었는데 반팔은 괜찮네요! 기본티로 참 적당해요\n길이도 넘 길지도 짧지도 않아서 여기저기 입기에 좋을 것 같아요~~~'}
3
{'selected_color': '핑크(세미크롭ver.)', 'selected_size': 'F(44~66)', 'size_opinion': '정사이즈예요', 'quality_opinion': '보통이에요', 'color_opinion': '화면과 비슷해요', 'height': '164cm', 'weight': '65kg', 'top_size': '상의 66', 'detail_text': '(키는 164cm 골격 등 체격이 좀 있는 편입니다.) 너무 길지도 않고 

## 색상 크롤링 테스트

In [85]:
color_string_tag = {'색상', 'color'}

def print_all_tag(ele):
    for e in ele:
        time.sleep(0.1)
        print(e.text)

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    color_set = set()
    driver.get('https://zigzag.kr/catalog/products/132082569')
    wait = WebDriverWait(driver, 5)
    buy_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'BODY_17.BOLD.css-hbld7k.e1yh52zv0')))
    ActionChains(driver).click(buy_button).perform()
    toggle_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'BODY_15.SEMIBOLD.css-utqis4.e1cn5bmz0')))
    tag_string = toggle_tag.text.lower()
    tag_list = driver.find_element(By.CLASS_NAME, 'css-0.e1u2d7n04').find_elements(By.TAG_NAME, 'li')
    if tag_string in color_string_tag:
        for color_tag in tag_list:
            time.sleep(0.1)
            color = color_tag.text.split('\n')[0]
            color_set.add(color)
    else:
        for i in range(len(tag_list)):
            time.sleep(0.1)
            tag = tag_list[i]
            tag.click()
            color_tag_list = driver.find_element(By.CLASS_NAME, 'css-0.e1u2d7n04').find_elements(By.TAG_NAME, 'li')
            for color_tag in color_tag_list:
                time.sleep(0.1)
                color = color_tag.text.split('\n')[0]
                color_set.add(color)
            tag_button = driver.find_element(By.CLASS_NAME, 'css-oa28ah.e1u2d7n06')
            tag_button.click()
            tag_list = driver.find_element(By.CLASS_NAME, 'css-0.e1u2d7n04').find_elements(By.TAG_NAME, 'li')
            driver.implicitly_wait(1)


print(color_set)


롱
숏(-1000)
롱
숏(-1000)
{'아이보리', '소라(숏단독컬러)', '차콜', '블루', '블랙', '브라운(롱단독컬러)', '베이지', '핑크'}


In [5]:
import pandas as pd

def set_rank(df, sorted_product_list):
    # 우선 기존의 rank 컬럼을 'none'으로 초기화
    df['rank'] = 'none'
    
    # sorted_product_list를 통해 rank를 업데이트
    rank_dict = {product_id: rank + 1 for rank, product_id in enumerate(sorted_product_list)}
    
    # rank_dict를 사용하여 df의 rank 컬럼 업데이트
    df['rank'] = df['product_id'].map(rank_dict).fillna('none')
    
    return df

# 예시 데이터프레임 생성
data = {
    'product_id': ['product_id_1', 'product_id_2', 'product_id_3', 'product_id_4'],
    'category': ['A', 'B', 'C', 'D'],
    'description': ['desc1', 'desc3', 'desc2', 'desc4'],
    'product_name': ['name1', 'name3', 'name2', 'name4'],
    'price': [100, 300, 200, 400],
    'image_url': ['url1', 'url3', 'url2', 'url4'],
    'size': ['S', 'M', 'L', 'XL'],
    'color': ['red', 'blue', 'green', 'yellow'],
    'rank': [1, 2, 3, 4]
}

df = pd.DataFrame(data)
print(df)

# sorted_product_list 예시
sorted_product_list = ['product_id_2', 'product_id_1', 'product_id_4', 'product_id_3']

# set_rank 메서드 호출
updated_df = set_rank(df, sorted_product_list)
print(updated_df)

     product_id category description product_name  price image_url size  \
0  product_id_1        A       desc1        name1    100      url1    S   
1  product_id_2        B       desc3        name3    300      url3    M   
2  product_id_3        C       desc2        name2    200      url2    L   
3  product_id_4        D       desc4        name4    400      url4   XL   

    color  rank  
0     red     1  
1    blue     2  
2   green     3  
3  yellow     4  
     product_id category description product_name  price image_url size  \
0  product_id_1        A       desc1        name1    100      url1    S   
1  product_id_2        B       desc3        name3    300      url3    M   
2  product_id_3        C       desc2        name2    200      url2    L   
3  product_id_4        D       desc4        name4    400      url4   XL   

    color  rank  
0     red     2  
1    blue     1  
2   green     4  
3  yellow     3  
